# Connect Playlists and Locations

In [1]:
import pandas as pd
import time
import json
import googlemaps
import homebrew as hb
from sklearn.neighbors import NearestNeighbors

In [74]:
%load_ext autoreload
%autoreload 2

In [3]:
df = pd.read_json('artists_shows.json', lines=True)

In [4]:
print(df.shape)
df.head()

(134943, 3)


,artist,date,loc
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US"
1,2 Chainz,2019-07-12 11:00:00,"London, UK"
2,2 Chainz,2019-08-10 00:00:00,"Anaheim, CA, US"
3,2 Chainz,2019-08-24 22:00:00,"Atlanta, GA, US"
4,2 Chainz,2019-05-27 05:30:00,"Las Vegas, NV, US"


In [5]:
df.date.describe()

count                  134943
unique                  37136
top       2019-07-19 00:00:00
freq                      191
first     1973-09-29 00:00:00
last      2020-12-16 18:30:00
Name: date, dtype: object

In [6]:
df.columns = ['artist', 'date', 'location']    # rename loc -> location
domestics = df[df['location'].str.endswith('US')]    # drop any foreign gigs
recent_domestics = domestics[domestics['date'] >= '2015']    #  drop any gigs older than 5 yrs
locations = recent_domestics.copy()

In [7]:
unique_locations = locations['location'].unique()
f'There are {unique_locations.shape[0]} unique locations'

'There are 2023 unique locations'

Get lat longs from googlemaps

In [8]:
# unique_loc_w_latlong = []

# for i, location in enumerate(unique_locations):
#     row = {}
#     lat, lng = hb.get_latlong(location)
#     row['location'] = location
#     row['lat'] = lat
#     row['lng'] = lng
#     unique_loc_w_latlong.append(row)
#     if i % 50 == 0:
#         time.sleep(2)
#     if i % 100 == 0:
#         print(i)                # Slowish 

In [9]:
latlongs_df = pd.DataFrame(unique_loc_w_latlong)
latlongs_df.head()

NameError: name 'unique_loc_w_latlong' is not defined

In [61]:
shows_w_latlong = pd.merge(locations, latlongs_df, on='location', how='left')
print(locations.shape)
shows_w_latlong.shape

(75699, 3)


(75699, 5)

In [62]:
shows_w_latlong.to_json('shows_w_latlong.json')

### Look for Similar Artists

In [94]:
shows_w_latlong = pd.read_json('shows_w_latlong.json')

In [95]:
artists_playlists = pd.read_json('trimmed_artists_w_flags.json', lines=True)

In [96]:
artists_playlists['artist'] = artists_playlists['artist'].map(str.lower)
shows_w_latlong['artist'] = shows_w_latlong['artist'].map(str.lower)

In [98]:
similar_artist_model = hb.SimilarArtistModel(artists_playlists)

In [99]:
main_artist = input().lower()
print(main_artist)
num_artists = 1
_, *similar_artists = similar_artist_model.find_artists_sim_to(main_artist, n_neighbors=num_artists+1)

for artist_tup in similar_artists:
    artist, cosine_sim = artist_tup
    print(f'Similar Band: {artist.upper()} - - Cosine Similarity: {cosine_sim}')
    new_spots = hb.pilfer_similar_artist(ref_df=shows_w_latlong,
                                         main_artist=main_artist,
                                         similar_artist=artist,
                                         max_new_cities=10)
    for spot in new_spots:
        print('\t', spot)

 Jason aldean


jason aldean
      artist
0   (g)i-dle
1     *nsync
2   2 chainz
3  21 savage
4       2pac
Similar Band: LUKE COMBS - - Cosine Similarity: 0.366
	 {'new': 'Palmer, AK, US', 'nearest_old': 'Puyallup, WA, US', 'distance': 1449.0}
	 {'new': 'Bozeman, MT, US', 'nearest_old': 'Spokane, WA, US', 'distance': 332.0}
	 {'new': 'Nampa, ID, US', 'nearest_old': 'Spokane, WA, US', 'distance': 285.0}
	 {'new': 'Lubbock, TX, US', 'nearest_old': 'Arlington, TX, US', 'distance': 281.0}
	 {'new': 'Duluth, MN, US', 'nearest_old': 'Oshkosh, WI, US', 'distance': 257.0}
	 {'new': 'Phoenix, AZ, US', 'nearest_old': 'Indio, CA, US', 'distance': 239.0}
	 {'new': 'St. Paul, MN, US', 'nearest_old': 'Oshkosh, WI, US', 'distance': 233.0}
	 {'new': 'Sioux Falls, SD, US', 'nearest_old': 'Des Moines, IA, US', 'distance': 208.0}
	 {'new': 'Charleston, SC, US', 'nearest_old': 'Charlotte, NC, US', 'distance': 177.0}
	 {'new': 'Missoula, MT, US', 'nearest_old': 'Spokane, WA, US', 'distance': 170.0}


In [76]:
main_artist = input()
num_artists = 3
_, *similar_artists = similar_artist_model.find_artists_sim_to(main_artist, n_neighbors=num_artists+1)

for artist_tup in similar_artists:
    artist, cosine_sim = artist_tup
    print(f'Similar Band: {artist.upper()} - - Cosine Similarity: {cosine_sim}')
    new_spots = hb.pilfer_similar_artist(ref_df=shows_w_latlong,
                                         main_artist=main_artist,
                                         similar_artist=artist,
                                         max_new_cities=3)
    for spot in new_spots:
        
        print('\t', spot)

 Jason Aldean


      artist
0   (G)I-DLE
1     *NSYNC
2   2 Chainz
3  21 Savage
4       2Pac
Similar Band: LUKE COMBS - - Cosine Similarity: 0.366
	 {'new': 'Palmer, AK, US', 'nearest_old': 'Puyallup, WA, US', 'distance': 1449.0}
	 {'new': 'Bozeman, MT, US', 'nearest_old': 'Spokane, WA, US', 'distance': 332.0}
	 {'new': 'Nampa, ID, US', 'nearest_old': 'Spokane, WA, US', 'distance': 285.0}
Similar Band: JORDAN DAVIS - - Cosine Similarity: 0.402
	 {'new': 'Fargo, ND, US', 'nearest_old': 'Des Moines, IA, US', 'distance': 398.0}
	 {'new': 'Bemidji, MN, US', 'nearest_old': 'Oshkosh, WI, US', 'distance': 387.0}
	 {'new': 'Aberdeen, SD, US', 'nearest_old': 'Des Moines, IA, US', 'distance': 362.0}
Similar Band: OLD DOMINION - - Cosine Similarity: 0.423
	 {'new': 'Honolulu, HI, US', 'nearest_old': 'Mountain View, CA, US', 'distance': 2405.0}
	 {'new': 'Kahului, HI, US', 'nearest_old': 'Mountain View, CA, US', 'distance': 2349.0}
	 {'new': 'Bismarck, ND, US', 'nearest_old': 'Cheyenne, WY, US', 'distance': 440.